In [5]:
import os
import requests
import pandas as pd

> Powerbank 

In [ ]:
import requests
import pandas as pd
import time
import random
import os
from urllib.parse import urljoin

# Configuration
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'en-US,en;q=0.9',
    'Referer': 'https://tiki.vn/sac-dung-phong/c1821',
}
IMAGE_DIR = 'product_images'
MAX_PAGES = 3
PRODUCTS_PER_PAGE = 17
DELAY_RANGE = (1.0, 2.5)

def setup_image_directory():
    os.makedirs(IMAGE_DIR, exist_ok=True)

def download_and_convert_image(image_url, product_id):
    if not image_url or not product_id:
        return ""
    
    try:
        # Standardize URL
        if image_url.startswith('//'):
            image_url = 'https:' + image_url
        
        # Set filename
        filename = f"{product_id}.jpg"
        filepath = os.path.join(IMAGE_DIR, filename)
        
        # Download image
        response = requests.get(image_url, headers=HEADERS, stream=True, timeout=10)
        response.raise_for_status()
        
        # Save as JPG
        with open(filepath, 'wb') as f:
            for chunk in response.iter_content(1024):
                f.write(chunk)
        
        return filename
        
    except Exception as e:
        print(f"⚠️ Failed to download image for product {product_id}: {str(e)}")
        return ""

def crawl_powerbanks_from_category():
    setup_image_directory()
    all_products = []
    category_name = "Power Banks"
    category_id = 1821

    for page in range(1, MAX_PAGES + 1):
        print(f"📊 Collecting page {page}/{MAX_PAGES}...")

        try:
            response = requests.get(
                "https://tiki.vn/api/v2/products",
                headers=HEADERS,
                params={
                    "limit": PRODUCTS_PER_PAGE,
                    "category": category_id,
                    "page": page,
                    "sort": "newest"
                },
                timeout=15
            )

            if response.status_code == 200:
                products = response.json().get("data", [])
                
                if not products:
                    print(f"⏩ No products on page {page}, stopping collection")
                    break

                for product in products:
                    product_id = str(product.get('id', ''))
                    thumbnail_url = product.get('thumbnail_url', '')
                    
                    image_filename = download_and_convert_image(thumbnail_url, product_id)
                    
                    all_products.append({
                        "product_id": product_id,
                        "category": category_name,
                        "product_name": product.get('name', ''),
                        "price": product.get('price', 0),
                        "image": image_filename,  # Saved filename instead of URL
                        "image_url": thumbnail_url,  # Keep original URL for reference
                        "source": f"https://tiki.vn/{product.get('url_path', '')}"
                    })

                print(f"✅ Added {len(products)} products from page {page}")
            else:
                print(f"⚠️ Error {response.status_code} on page {page}")
                break

        except Exception as e:
            print(f"❌ Connection error: {str(e)}")
            break

        time.sleep(random.uniform(*DELAY_RANGE))

    return all_products

if __name__ == "__main__":
    print("🛒 Starting to collect power banks from Tiki...")
    products = crawl_powerbanks_from_category()
    
    
    # Print sample product
    if products:
        sample = products[0]
        print("\nSample product:")
        print(f"ID: {sample['product_id']}")
        print(f"Name: {sample['product_name']}")
        print(f"Price: {sample['price']:,} VND")
        print(f"Image file: {sample['image']}")
        print(f"Image URL: {sample['image_url'][:50]}...")
        print(f"Link: {sample['source']}")

> SMART WATCH 

In [6]:
CATEGORY_ID = 8039
CATEGORY_NAME = "Smart Watches"
NUM_PAGES = 2
SAVE_DIR = "images/watch"
os.makedirs(SAVE_DIR, exist_ok=True)

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

records = []

# Function to download image
def download_image(url, save_path):
    try:
        response = requests.get(url, headers=HEADERS)
        if response.status_code == 200:
            with open(save_path, "wb") as f:
                f.write(response.content)
    except Exception as e:
        print(f"Error downloading image: {e}")

In [7]:
for page in range(1, NUM_PAGES + 1):
    api_url = f"https://tiki.vn/api/personalish/v1/blocks/listings?limit=40&category={CATEGORY_ID}&page={page}&sort_by=popularity"

    try:
        response = requests.get(api_url, headers=HEADERS)
        if response.status_code != 200:
            print(f"HTTP Error {response.status_code} on page {page}")
            continue

        data = response.json()
    except Exception as e:
        print("Unable to parse JSON:", e)
        print("Response:", response.text[:300])
        continue

    for item in data.get("data", []):
        product_id = item.get("id")
        name = item.get("name", "no_name")
        price = item.get("price", 0)
        image_url = item.get("thumbnail_url")

        if not image_url or not product_id:
            continue

        filename = f"{product_id}.jpg"
        full_path = os.path.join(SAVE_DIR, filename)
        download_image(image_url, full_path)

        records.append({
            "product_id": product_id,
            "category": CATEGORY_NAME,
            "product_name": name,
            "price": price,
            "image": filename,
            "source": image_url
        })

        print(f"Downloaded: {name}")


Downloaded: Dây Cáp Sạc Từ Tính cho Đồng Hồ Xiaomi Mi Band 5/ Mi Band 6/ Mi Band 7/ Mi band 8/ Mi band 9/ Mi band 10- Hàng chính hãng - Miband 5/6/7
Downloaded: Dây đeo cho vòng đeo tay thông minh Xiaomi MiBand 8/ Miband 9/ Mi band 10 ngàm thép_ Hàng Chính Hãng
Downloaded: Cáp sạc thay thế cho các dòng đồng hồ Garmin Fenix 7, Fenix 6, Fenix 5, Forerunner 955, 945, 935, 245, 265, 255, 45, 645, Instinct, Viviactive 3, Venu/ Garmin Epix Gen 2- Hàng Nhập Khẩu 
Downloaded: Vòng Đeo Tay Thông Minh Xiaomi Smart Band 9 Active – Hàng Chính Hãng
Downloaded: Dock Sạc Cho Huawei Watch GT 2 Pro / GT Runner / GT3/3Pro / GT4/4Pro / Huawei Watch GT5 / Huawei Watch GT 5 Pro - Hàng Chính Hãng
Downloaded: Đồng Hồ Thông Minh Xiaomi Redmi Watch 5 ACTIVE – Hàng Chính Hãng
Downloaded: Cáp Sạc Đế Sạc cho Samsung Galaxy Watch 3/4/4Classic/5/5Pro/6/6Classic/7/Ultra & Galaxy Watch Active 1/2- Hàng Chính Hãng
Downloaded: Apple Watch Series 10 GPS (Viền nhôm, dây cao su)
Downloaded: Apple Watch SE 2024 GPS (Viền n

In [10]:
df = pd.DataFrame(records)
df.to_excel("data.xlsx", index=False)